# EM Algorithm
Problem 3 homework 2

In [244]:
%load_ext autoreload
%autoreload 2
import numpy as np
import data_generator
import em_implementation
import utilities

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Build DataGenerator with True Parameters

Notes: smaller evaluations with B = 20 experiments tested several different arrangement of true parameters and weights.
Means that were far apart caused easy convergence, but also caused underflow if initial means were not chosen from
different starting components. Very close means cause issues even if there are significant differences in the sigmas.
the model still achieves moderate accuracy, but can have issues with underflow due to one of the sigmas becoming very
small. Reversing the order of the higher mean can cause the parameters to be
learned for the opposite components. The values still converge to good approximates but the order switches because the
centroids end up on the other sides. A range of weights combinations were also tried, with all cases successful.



In [261]:
MEANS = [1000, -20]
STANDARD_DEVIATIONS = [5, 0.25]

# weights for first of two components
ALPHA = 0.1
BETA = 0.9

gen = data_generator.DataGenerator(2,MEANS,STANDARD_DEVIATIONS)

## Evaluation 1: m, n = 100

Notes: The full data noticeably improves the accuracy and stability of estimates compared to the partial data.
This makes intuitive sense because there is effectively twice as much data. Also note the reduced variance for the
second component in the full data compared to the partial. This may be due to the greater amount of data or also
because the partial is only relying on information with a 0.8 weight towards component with the high variance estimates. 

## Evaluation 2: m, n = 1000
Notes: This larger trial further supports the conclusions from the previous evaluation. It also shows greater accuracy
and stability for both the partial and full data compared to the previous trial. This shows that larger datasets are
are also favorable for the partial training algorithm. This does not appear to make up for the issue with the heavily
weighted component in the partial data having higher variance in its estimates.

## Run the EM Algorithm!

Notes: various stoppage conditions were examined. Examining log likelihood proved problematic because logs of unlikely
individual data points can cause issues from underflow. By itself, simple stoppage after a fixed number of steps worked
well enough if the number of steps was sufficiently high. However this was computationally wasteful. By adding a check
to see if none of the parameters had updated, an earlier termination can be achieved. This did not appear to affect
accuracy, as differences in estimates from step to step never increased once reaching 0.0.

Also of interest, initial experiments used random selection to choose starting mean values. This produced very poor
results when both both means were chosen from the same component. As simple alternative, later experiments select the 
extrema of the dataset to ensure that the means are not close together and most likely represent both components. Further
research could investigate the effect of this choice on handling mostly overlapping distributions. Initial investigation
suggests it does no worse with this edge case than random selection.

In [263]:
m = 100
n = 100

np.random.seed(12345)

#Repeat experiment B times
B = 1000

mean_partial_results = []
standard_deviation_partial_results = []
beta_partial_results = []

mean_results = []
standard_deviation_results = []
alpha_results = []
beta_results = []

for i in range(B):
    # get data
    D_x = gen.get_data([ALPHA, 1-ALPHA], m)
    D_y = gen.get_data([BETA, 1-BETA], n)
    
    # estimate parameters and weights from just D_y
    means, standard_deviations, beta = em_implementation.compute_EM(D_y)
    
    mean_partial_results.append(means)
    standard_deviation_partial_results.append(standard_deviations)
    beta_partial_results.append(beta)
    
    # estimate parameters and weights from full data
    means, standard_deviations, alpha, beta = em_implementation.compute_EM_two_datasets(D_x, D_y)
    
    mean_results.append(means)
    standard_deviation_results.append(standard_deviations)
    alpha_results.append(alpha)
    beta_results.append(beta)


print('****** partial data results *******')
print('mu means:', utilities.get_sample_means(mean_partial_results))
print('mu variances:', utilities.get_sample_variance(mean_partial_results))

print('sigma means:', utilities.get_sample_means(standard_deviation_partial_results))
print('sigma variances:', utilities.get_sample_variance(standard_deviation_partial_results))

print('beta means:', utilities.get_sample_means(beta_partial_results))
print('beta variances:', utilities.get_sample_variance(beta_partial_results))


print('\n\n****** full data results *******')
print('mu means:', utilities.get_sample_means(mean_results))
print('mu variances:', utilities.get_sample_variance(mean_results))

print('sigma means:', utilities.get_sample_means(standard_deviation_results))
print('sigma variances:', utilities.get_sample_variance(standard_deviation_results))

print('alpha means:', utilities.get_sample_means(alpha_results))
print('alpha variances:', utilities.get_sample_variance(alpha_results))

print('beta means:', utilities.get_sample_means(beta_results))
print('beta variances:', utilities.get_sample_variance(beta_results))

****** partial data results *******
mu means: [999.9772292087641, -20.000184416733113]
mu variances: [0.28355629746937494, 0.006502750630676686]
sigma means: [4.939195169225679, 0.2256925585881193]
sigma variances: [0.12541971419509979, 0.0032181072888461454]
beta means: 0.8995700000000001
beta variances: 0.0008593151000000003


****** full data results *******
mu means: [999.9849608942054, -19.99954369833766]
mu variances: [0.24792122609370673, 0.0006216510364478071]
sigma means: [4.949536892581556, 0.2478074206026152]
sigma variances: [0.11034819033452403, 0.00028823227791763143]
alpha means: 0.10061000000000002
alpha variances: 0.0009521278999999998
beta means: 0.8995700000000001
beta variances: 0.0008593151000000003


## Additional experiments